# Kshitij Sonje CS24MTECH11025

# Assignment 2


In [36]:
import csv
import numpy as np
from scipy.linalg import null_space
import random

np.random.seed(0)
random.seed(0)

In [37]:
# Extract starting point and objective coefficients from CSV data
def extract_objective_data(raw_data):
    start_point = np.array([float(val) for val in raw_data[0][:-1]])
    cost_coeffs = np.array([float(val) for val in raw_data[1][:-1]])
    return start_point, cost_coeffs

# Extract constraint bounds from CSV data
def extract_constraint_bounds(raw_data):
    return np.array([float(row[-1]) for row in raw_data[2:]])

# Extract constraint coefficient matrix from CSV data
def extract_constraint_matrix(raw_data):
    return np.array([[float(val) for val in row[:-1]] for row in raw_data[2:]])

In [38]:
# Parse CSV file and return problem data
def parse_csv_input(filename, has_initial_point=True):
    with open(filename, newline='') as csvfile:
        reader = csv.reader(csvfile)
        raw_data = list(reader)
    
    start_point, cost_coeffs = extract_objective_data(raw_data)
    bounds = extract_constraint_bounds(raw_data)
    constraint_mat = extract_constraint_matrix(raw_data)
    
    return constraint_mat, start_point, bounds, cost_coeffs

In [39]:
# Compute active constraint mask based on tolerance
def compute_constraint_products(coeff_matrix, current_pt, bounds, tolerance=1e-8):
    constraint_vals = np.dot(coeff_matrix, current_pt)
    active_mask = np.abs(constraint_vals - bounds) < tolerance
    return active_mask

# Separate constraints into active and inactive sets
def separate_constraint_sets(coeff_matrix, active_mask):
    active_constraints = coeff_matrix[active_mask]
    inactive_constraints = coeff_matrix[~active_mask]
    return active_constraints, inactive_constraints

In [40]:
# Analyze constraints and return active/inactive sets
def analyze_constraints(coeff_matrix, current_pt, bounds, tolerance=1e-8):
    active_mask = compute_constraint_products(coeff_matrix, current_pt, bounds, tolerance)
    active_set, inactive_set = separate_constraint_sets(coeff_matrix, active_mask)
    return active_mask, active_set, inactive_set

In [41]:
# Invert matrix and negate for movement directions
def invert_and_negate_matrix(active_matrix):
    matrix_inverse = np.linalg.inv(active_matrix)
    negated_inverse = -1 * matrix_inverse
    return negated_inverse

# Handle singular matrix cases
def handle_singular_matrix():
    return None


In [42]:
# Compute movement direction vectors
def compute_movement_vectors(active_matrix):
    try:
        return invert_and_negate_matrix(active_matrix)
    except np.linalg.LinAlgError:
        return handle_singular_matrix()

In [43]:
# Check if constraint matrix is degenerate
def is_degenerate(active_constraint_matrix):
    if len(active_constraint_matrix) == 0:
        return False
    
    rows, cols = active_constraint_matrix.shape
    
    if rows > cols:
        return True
    try:
        rank = np.linalg.matrix_rank(active_constraint_matrix)
        if rank < min(rows, cols):
            return True
    except np.linalg.LinAlgError:
        return True
    return False

In [44]:
# Apply epsilon perturbation to constraint bounds
def apply_perturbation_terms(original_bounds, epsilon_val):
    adjusted_epsilon = epsilon_val
    perturbed_bounds = np.array([
        original_bounds[idx] + adjusted_epsilon**(idx+1) 
        for idx in range(len(original_bounds))
    ])
    return perturbed_bounds, adjusted_epsilon

# Remove degeneracy using perturbation technique
def remove_degeneracy_condition(original_constraint_bounds, epsilon_value, shrink_rate=0.5):
    reduced_epsilon = epsilon_value * shrink_rate
    modified_bounds, final_epsilon = apply_perturbation_terms(
        original_constraint_bounds, 
        reduced_epsilon
    )
    return modified_bounds, final_epsilon

In [45]:
# Initialize iteration variables
def setup_iteration_variables(start_point):
    current_position = start_point
    step_count = 0
    display_frequency = 1
    return current_position, step_count, display_frequency

# Execute vertex search iterations with cycling detection
def execute_vertex_search_iterations(constraint_matrix, bounds_vector, objective_vector, problem_dimension, current_pos, step_count, display_freq, cost_history, position_history):
    max_iterations = 100
    active_mask, active_set, inactive_set = analyze_constraints(constraint_matrix, current_pos, bounds_vector)
    current_rank = calculate_matrix_rank(active_set)
    iteration_idx = 0
    recent_positions = []
    max_history = 10
    
    while iteration_idx < max_iterations and current_rank != problem_dimension:
        step_count += 1
        for prev_pos in recent_positions:
            if np.allclose(current_pos, prev_pos, atol=1e-10):
                return current_pos, active_set, cost_history, position_history
        
        recent_positions.append(current_pos.copy())
        if len(recent_positions) > max_history:
            recent_positions.pop(0)
        
        progress_info = show_iteration_progress(step_count, current_rank, display_freq)
        if progress_info[1] != display_freq:
            display_freq = progress_info[1]
        
        movement_direction = determine_search_direction(active_set, inactive_set)
        step_size = compute_step_magnitude(bounds_vector, active_mask, inactive_set, current_pos, movement_direction)
        
        if step_size <= 1e-12:
            break
            
        current_pos, active_mask, active_set, inactive_set, current_rank = perform_vertex_update(
            constraint_matrix, bounds_vector, current_pos, step_size, movement_direction, objective_vector, cost_history, position_history)
        
        iteration_idx += 1
    
    return current_pos, active_set, cost_history, position_history

# Show iteration progress with reduced output
def show_iteration_progress(current_step, matrix_rank, freq):
    should_display = current_step % 50 == 0
    new_freq = freq
    return should_display, new_freq

# Calculate step size magnitude
def compute_step_magnitude(bounds_vec, constraint_mask, inactive_constraints, current_point, direction):
    original_direction = direction.copy()
    max_attempts = 2
    
    for attempt in range(max_attempts):
        step_ratios = []
        for bound_val, constraint_row in zip(bounds_vec[~constraint_mask], inactive_constraints):
            denominator = np.dot(constraint_row, direction)
            if abs(denominator) > 1e-10:
                ratio = (bound_val - np.dot(constraint_row, current_point)) / denominator
                step_ratios.append(ratio)
        
        valid_steps = [ratio for ratio in step_ratios if ratio > 1e-12]
        
        if len(valid_steps) > 0:
            optimal_step = min(valid_steps)
            return optimal_step
        elif attempt == 0:
            direction = -1 * direction
        else:
            return 0.0
    
    return 0.0

# Update vertex position and related variables
def perform_vertex_update(coeff_matrix, bounds_vec, old_position, step_size, move_direction, cost_vector, cost_track, position_track):
    new_position = old_position + step_size * move_direction
    updated_mask, updated_active, updated_inactive = analyze_constraints(coeff_matrix, new_position, bounds_vec)
    updated_rank = calculate_matrix_rank(updated_active)
    cost_track.append(np.dot(cost_vector, new_position))
    position_track.append(new_position)
    return new_position, updated_mask, updated_active, updated_inactive, updated_rank

# Calculate matrix rank
def calculate_matrix_rank(constraint_matrix):
    if len(constraint_matrix) == 0:
        return 0
    else:
        return np.linalg.matrix_rank(constraint_matrix)

# Determine search direction using null space or random
def determine_search_direction(active_constraints, inactive_constraints):
    if len(active_constraints) == 0:
        np.random.seed(42)
        direction_vector = np.random.rand(inactive_constraints.shape[-1])
    else:
        null_space_matrix = null_space(active_constraints)
        if null_space_matrix.shape[1] == 0:
            np.random.seed(42)
            direction_vector = np.random.rand(active_constraints.shape[1])
        else:
            direction_vector = null_space_matrix[:, 0]
    return direction_vector

# Transform feasible point to vertex
def transform_feasible_to_vertex(constraint_matrix, bounds_vector, initial_point, cost_vector, problem_dimensions):
    cost_history = []
    position_history = []
    cost_history.append(np.dot(cost_vector, initial_point))
    position_history.append(initial_point)

    current_pos, step_count, display_freq = setup_iteration_variables(initial_point)
    active_mask, active_set, inactive_set = analyze_constraints(constraint_matrix, initial_point, bounds_vector)
    matrix_rank = calculate_matrix_rank(active_set)

    if matrix_rank == problem_dimensions:
        return current_pos, cost_history, position_history

    final_pos, final_active_set, final_cost_history, final_position_history = execute_vertex_search_iterations(
        constraint_matrix, bounds_vector, cost_vector, problem_dimensions, 
        current_pos, step_count, display_freq, cost_history, position_history)

    if not(is_degenerate(final_active_set)):
        return (final_pos, final_cost_history, final_position_history)
    else:
        return (None,)

In [46]:
# Setup vertex optimization variables
def setup_vertex_optimization(starting_vertex, cost_coefficients):
    current_vertex = starting_vertex
    next_vertex = current_vertex
    cost_trajectory = []
    vertex_trajectory = []
    cost_trajectory.append(np.dot(cost_coefficients, current_vertex))
    vertex_trajectory.append(current_vertex)
    step_counter = 0
    output_frequency = 1
    return current_vertex, next_vertex, cost_trajectory, vertex_trajectory, step_counter, output_frequency

# Execute optimization iterations
def execute_optimization_iterations(constraint_matrix, bounds_vector, initial_vertex, cost_vector, max_steps=100):
    current_vertex, next_vertex, cost_trajectory, vertex_trajectory, step_counter, output_frequency = setup_vertex_optimization(initial_vertex, cost_vector)
    
    for iteration_count in range(max_steps):
        step_counter += 1
        display_progress_information(step_counter, output_frequency)
        constraint_mask, active_constraints, inactive_constraints = analyze_constraints(constraint_matrix, current_vertex, bounds_vector)
        
        if is_degenerate(active_constraints):
            return None, cost_trajectory, vertex_trajectory
        
        movement_vectors = compute_movement_vectors(active_constraints)
        if movement_vectors is None:
            return None, cost_trajectory, vertex_trajectory
        
        movement_options = movement_vectors.T
        beneficial_directions = identify_beneficial_directions(movement_options, cost_vector)
        
        if not beneficial_directions:
            return next_vertex, cost_trajectory, vertex_trajectory
        
        selected_direction = beneficial_directions[0]
        step_length = calculate_optimization_step_size(bounds_vector, constraint_mask, inactive_constraints, current_vertex, selected_direction)
        
        if step_length is None:
            return None, cost_trajectory, vertex_trajectory
        
        current_vertex, next_vertex = update_vertex_position(current_vertex, step_length, selected_direction, cost_vector, cost_trajectory, vertex_trajectory)
    
    return None, cost_trajectory, vertex_trajectory

# Display progress information
def display_progress_information(current_step, display_freq):
    pass

# Identify beneficial movement directions
def identify_beneficial_directions(direction_matrix, objective_coefficients):
    improving_moves = []
    direction_index = 0
    
    while direction_index < len(direction_matrix):
        current_direction = direction_matrix[direction_index]
        improvement_measure = np.dot(current_direction, objective_coefficients)
        if improvement_measure > 0:
            improving_moves.append(current_direction)
        direction_index += 1
    
    return improving_moves

# Calculate optimization step size
def calculate_optimization_step_size(bounds_vec, active_mask, inactive_set, current_position, move_direction):
    step_options = []
    for bound_value, constraint_row in zip(bounds_vec[~active_mask], inactive_set):
        denominator = np.dot(constraint_row, move_direction)
        if abs(denominator) > 1e-10:
            ratio = (bound_value - np.dot(constraint_row, current_position)) / denominator
            step_options.append(ratio)
    
    valid_step_sizes = [step for step in step_options if step > 0]
    
    if len(valid_step_sizes) == 0:
        return None
    
    optimal_step_length = min(valid_step_sizes)
    return optimal_step_length

# Update vertex position
def update_vertex_position(old_vertex, step_size, direction, cost_coeffs, cost_track, vertex_track):
    new_vertex = old_vertex + step_size * direction
    updated_vertex = new_vertex
    cost_track.append(np.dot(cost_coeffs, new_vertex))
    vertex_track.append(new_vertex)
    return updated_vertex, updated_vertex

# Optimize from vertex to vertex
def optimize_vertex_to_vertex(constraint_matrix, bounds_vector, starting_vertex, objective_coefficients, problem_dimensions, iteration_limit=100):
    result = execute_optimization_iterations(constraint_matrix, bounds_vector, starting_vertex, objective_coefficients, max_steps=iteration_limit)
    return result

In [ ]:
constraint_matrix, initial_point, bounds_vector, objective_coefficients = parse_csv_input('Testcase.csv', has_initial_point=True)

problem_rows, problem_dimensions = len(bounds_vector), len(objective_coefficients)

print("Starting Feasible Point:", initial_point)
print("Objective Function Coefficients:", objective_coefficients)
print("Constraint Bounds Vector:", bounds_vector)
print("Constraint Coefficient Matrix:")
print(constraint_matrix)
print(f"Problem size: {problem_rows} constraints, {problem_dimensions} variables")

Starting Feasible Point: [0. 0.]
Objective Function Coefficients: [5. 2.]
Constraint Bounds Vector: [1. 1. 1.]
Constraint Coefficient Matrix:
[[ 0.  1.]
 [ 1.  0.]
 [-1. -1.]]
Problem size: 3 constraints, 2 variables


In [48]:
# Execute phase 1 feasible to vertex search
def execute_phase_one_search(constraint_matrix, bounds_vector, initial_point, objective_coefficients, problem_dimensions, modified_bounds):
    phase1_results = transform_feasible_to_vertex(constraint_matrix, modified_bounds, initial_point, objective_coefficients, problem_dimensions)
    return phase1_results

# Execute phase 2 vertex to vertex optimization
def execute_phase_two_optimization(constraint_matrix, bounds_vector, vertex_position, objective_coefficients, problem_dimensions, iteration_limit, modified_bounds):
    phase2_results = optimize_vertex_to_vertex(constraint_matrix, modified_bounds, vertex_position, objective_coefficients, problem_dimensions, iteration_limit)
    return phase2_results

In [49]:
# Process and extract final optimization results
def process_final_results(phase2_results, objective_coefficients):
    if phase2_results[0] is None:
        return None, phase2_results[1] if len(phase2_results) > 1 else [], phase2_results[2] if len(phase2_results) > 2 else []
    else:
        optimal_vertex, phase2_costs, phase2_trajectory = phase2_results
        return optimal_vertex, phase2_costs, phase2_trajectory

In [50]:
# Display vertex marching sequence and final results
def display_vertex_sequence(phase1_trajectory, phase1_costs, phase2_trajectory, phase2_costs, optimal_vertex, objective_coefficients):
    print("\nVERTEX MARCHING SEQUENCE:")
    all_vertices = phase1_trajectory + (phase2_trajectory[1:] if len(phase2_trajectory) > 1 else [])
    all_costs = phase1_costs + (phase2_costs[1:] if len(phase2_costs) > 1 else [])
    
    for step, (vertex, cost) in enumerate(zip(all_vertices, all_costs), 1):
        print(f"Step {step}: Vertex → {vertex} (Cost: {cost:.4f})")
    
    if optimal_vertex is not None:
        final_cost = np.dot(objective_coefficients, optimal_vertex)
        print(f"\nOptimal Solution: {optimal_vertex} with cost {final_cost:.4f}")
    else:
        print("\nResult: The optimization problem is unbounded!")

In [51]:
iteration_limit = 100
perturbation_value = 0.1
modified_bounds = bounds_vector
max_attempts = 5

for attempt_num in range(max_attempts + 1):
    if attempt_num > 0:
        modified_bounds, perturbation_value = remove_degeneracy_condition(bounds_vector, perturbation_value)

    # Phase 1: Find initial vertex
    phase1_results = execute_phase_one_search(constraint_matrix, bounds_vector, initial_point, objective_coefficients, problem_dimensions, modified_bounds)
    if len(phase1_results) == 1:
        if attempt_num >= max_attempts:
            break
        continue

    vertex_position, phase1_costs, phase1_trajectory = phase1_results

    # Phase 2: Optimize from vertex
    phase2_results = execute_phase_two_optimization(constraint_matrix, bounds_vector, vertex_position, objective_coefficients, problem_dimensions, iteration_limit, modified_bounds)
    if len(phase2_results) == 1:
        if attempt_num >= max_attempts:
            break
        continue

    # Process final results
    optimal_vertex, phase2_costs, phase2_trajectory = process_final_results(phase2_results, objective_coefficients)
    
    # Display results
    display_vertex_sequence(phase1_trajectory, phase1_costs, phase2_trajectory, phase2_costs, optimal_vertex, objective_coefficients)
    break


VERTEX MARCHING SEQUENCE:
Step 1: Vertex → [0. 0.] (Cost: 0.0000)
Step 2: Vertex → [0.39395654 1.        ] (Cost: 3.9698)
Step 3: Vertex → [-2.  1.] (Cost: -8.0000)
Step 4: Vertex → [ 1. -2.] (Cost: 1.0000)
Step 5: Vertex → [1. 1.] (Cost: 7.0000)

Optimal Solution: [1. 1.] with cost 7.0000
